In [1]:
# Committing and Rolling Back a Transaction


In [2]:
import mysql.connector as mysql
import os 
from dotenv import load_dotenv

In [ ]:
!docker stop mysql-container
!docker restart mysql-container
!docker ps

In [3]:
load_dotenv('/workspaces/IBM-DS-Course/.env')
user=os.getenv('USER')
password=os.getenv('PASSWORD')
host = 'localhost'
port=3306


In [ ]:
conn = mysql.connect(
    host=host,
    user=user,
    password=password,
    port=port
)
db = 'Transaction'
cursor = conn.cursor()
create_db =f'CREATE DATABASE IF NOT EXISTS {db}'
cursor.execute(create_db)

In [ ]:
sql_scripts = ['/workspaces/IBM-DS-Course/Course 6 Db and SQL /Module 6 /1.3 .sql/BankAccounts-CREATE.sql', '/workspaces/IBM-DS-Course/Course 6 Db and SQL /Module 6 /1.3 .sql/ShoeShop-CREATE.sql']
for sql_script in sql_scripts:
    with open(sql_script, 'r') as script:
        script_cmd = script.read()

use_db = f'USE {db}'
cursor.execute(use_db)

try:
    for cmd in script_cmd.split(";"):
        if cmd.strip():
            cursor.execute(cmd)
        conn.commit()
        print("Success")
except Exception as e:
    print(f'{cmd} failed because e')
    
cursor.close()

## Sample Exercise


1. Example of committing and rolling back a transaction.



- Scenario: Rose is buying a pair of boots from ShoeShop. So we have to update Rose's balance as well as the ShoeShop balance in the BankAccounts table. 
- Then we also have to update Boots stock in the ShoeShop table. 
- After Boots, let's also attempt to buy Rose a pair of Trainers.


2. Once the tables are ready, create a stored procedure routine named TRANSACTION_ROSE that includes TCL commands like COMMIT and ROLLBACK.


- Now develop the routine based on the given scenario to execute a transaction.
- To create the stored procedure routine on MySQL, copy the code below and paste it to the textarea of the SQL page. Click Go.

In [ ]:
'''CREATE PROCEDURE TRANSACTION_ROSE()
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        RESIGNAL;
    END;
    START TRANSACTION;
    UPDATE BankAccounts
    SET Balance = Balance-200
    WHERE AccountName = 'Rose';
    UPDATE BankAccounts
    SET Balance = Balance+200
    WHERE AccountName = 'Shoe Shop';
    UPDATE ShoeShop
    SET Stock = Stock-1
    WHERE Product = 'Boots';
    UPDATE BankAccounts
    SET Balance = Balance-300
    WHERE AccountName = 'Rose';
    COMMIT;
END; '''

3. Let's now check if the transaction can successfully be committed or not. Copy the code below in a new blank script and paste it to the textarea of the SQL page. Click Go


In [ ]:
'''CALL TRANSACTION_ROSE;'''
'''SELECT * FROM BankAccounts;'''
'''SELECT * FROM ShoeShop;'''

4. Observe that the transaction has been executed. But when we observe the tables, no changes have permanently been saved through COMMIT. All the possible changes happened might have been undone through ROLLBACK since the whole transaction fails due to the failure of a SQL statement or more. Let's go through the possible reason behind the failure of the transaction and how COMMIT - ROLLBACK works on a stored procedure:

The first three UPDATEs should run successfully. Both the balance of Rose and ShoeShop should have been updated in the BankAccounts table. The current balance of Rose should stand at 300 - 200 (price of a pair of Boots) = 100. The current balance of ShoeShop should stand at 124,200 + 200 = 124,400. The stock of Boots should also be updated in the ShoeShop table after the successful purchase for Rose, 11 - 1 = 10.

The last UPDATE statement tries to buy Rose a pair of Trainers, but her balance becomes insufficient (Current balance of Rose: 100 < Price of Trainers: 300) after buying a pair of Boots. So, the last UPDATE statement fails. Since the whole transaction fails if any of the SQL statements fail, the transaction won't be committed.

Previous


## Practice exercise


Create a stored procedure TRANSACTION_JAMES to execute a transaction based on the following scenario: 
- First buy James 4 pairs of Trainers from ShoeShop:

- - Update his balance as well as the balance of ShoeShop. 
- - Also, update the stock of Trainers at ShoeShop. 

- Then attempt to buy James a pair of Brogues from ShoeShop. 
- - If any of the UPDATE statements fail, the whole transaction fails. 
- - You will roll back the transaction. 

- Commit the transaction only if the whole transaction is successful.

In [ ]:
'''CREATE PROCEDURE TRANSACTION_JAMES()
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        RESIGNAL;
    END;
    START TRANSACTION;
    UPDATE BankAccounts
    SET Balance = Balance-1200
    WHERE AccountName = 'James';
    UPDATE BankAccounts
    SET Balance = Balance+1200
    WHERE AccountName = 'Shoe Shop';
    UPDATE ShoeShop
    SET Stock = Stock-4
    WHERE Product = 'Trainers';
    UPDATE BankAccounts
    SET Balance = Balance-150
    WHERE AccountName = 'James';
    COMMIT;
END '''